In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T

import pickle


In [2]:
# Device configuration

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('device:', device)

device: cuda


In [3]:
from IPython.display import HTML, display


# Custom IPython progress bar for training
class ProgressMonitor(object):

    tmpl = """
        <table style="width: 100%;">
            <tbody>
                <tr>
                    <td style="width: 30%;">
                     <b>Loss: {loss:0.4f}</b> &nbsp&nbsp&nbsp {value} / {length}
                    </td>
                    <td style="width: 70%;">
                        <progress value='{value}' max='{length}', style='width: 100%'>{value}</progress>
                    </td>
                </tr>
            </tbody>
        </table>
        """

    def __init__(self, length):
        self.length = length
        self.count = 0
        self.display = display(self.html(0, 0), display_id=True)

    def html(self, count, loss):
        return HTML(self.tmpl.format(length=self.length, value=count, loss=loss))

    def update(self, count, loss):
        self.count += count
        self.display.update(self.html(self.count, loss))

In [4]:
transform_train = T.Compose( [T.RandomCrop(32, padding=4), T.ToTensor(), T.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) )] )
transform_test = T.Compose( [T.ToTensor(), T.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) )] )

train_set = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train )
test_set = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test )

classes = train_set.classes

100%|██████████| 170498071/170498071 [00:02<00:00, 58119999.83it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
print(train_set.data.shape)
print(test_set.data.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [11]:
# 과제 1- SimpleCNN의 오류를 없애라!
# Hint- matrix의 size를 주의! maxpooling은 size를 1/2배한다. filter의 size도 중요

class SimpleCNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv_layers = nn.Sequential(


            #### 입력 이미지 크기 32x32x3

            nn.Conv2d( in_channels=3, out_channels=64, kernel_size=2, padding=1 ),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d( in_channels=64, out_channels=64, kernel_size=2, padding=1 ),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.MaxPool2d(2),

            nn.Conv2d( in_channels=64, out_channels=128, kernel_size=2, padding=1 ),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.MaxPool2d(2),


        )

        self.fc_layers = nn.Sequential(

            nn.Linear( 128 * 9 * 9, 500),
            nn.ReLU(),

            nn.Linear(500, 10),

        )


    def forward(self, x):

        x = self.conv_layers(x)

        x = x.view( x.size(0), -1 ) # flatten

        x = self.fc_layers(x)

        return x

In [12]:
# 모델 테스트
# 텐서의 사이즈가 (7, 10)이 나오면 성공
# 현재는 오류가 뜨는 상황! matrix size를 잘 맞춰서 이 코드가 정상적으로 구동되면 성공입니다.

temp = SimpleCNN()
output = torch.randn( 7, 3, 32, 32)

print( temp(output).size() )


torch.Size([7, 10])


In [25]:
# 실습 2- Resnet 구현(선택)
# Hint- layer를 지나간 뒤 input을 더해주어야 한다, stride말고 maxpool로 size 줄여도 괜찮습니다.


In [26]:
# Resnet 모델 테스트
# 텐서의 사이즈가 (7, 10)이 나오면 성공

temp = Resnet()
output = torch.randn( 7, 3, 32, 32)

print( temp(output).size() )


torch.Size([7, 10])


In [14]:
batch_size = 128
learning_rate = 0.001
num_epochs = 50

In [15]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [16]:
# 원하는 모델을 돌려보세요

model = SimpleCNN()

model.to(device)

SimpleCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=10368, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=10, bias=True)
  )
)

In [17]:
# Loss Function
criterion = nn.CrossEntropyLoss()
# optimizer 선정
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [18]:
from statistics import mean

def train(optimizer, model, num_epochs=10, first_epoch=1):

    criterion = nn.CrossEntropyLoss()

    train_losses = []
    test_losses = []

    for epoch in range(first_epoch, first_epoch + num_epochs):
        print('Epoch', epoch)

        # train phase
        model.train()

        # create a progress bar
        progress = ProgressMonitor(length=len(train_set))

        # keep track of predictions
        correct_train = 0

        batch_losses = []

        for batch, targets in train_loader:

            # Move the training data to the GPU
            batch = batch.to(device)
            targets = targets.to(device)

            # clear previous gradient computation
            optimizer.zero_grad()

            # forward propagation
            outputs = model(batch)

            # calculate the loss
            loss = criterion(outputs, targets)

            # backpropagate to compute gradients
            loss.backward()

            # update model weights
            optimizer.step()

            batch_losses.append(loss.item())

            # accumulate correct count
            _, preds = torch.max(outputs, 1)
            correct_train += torch.sum(preds == targets.data)

            # update progress bar
            progress.update(batch.shape[0], mean(batch_losses) )


        train_losses.append( mean(batch_losses))


        # test phase
        model.eval()

        y_pred = []

        correct_test = 0

        # We don't need gradients for test, so wrap in
        # no_grad to save memory
        with torch.no_grad():

            for batch, targets in test_loader:

                # Move the training batch to the GPU
                batch = batch.to(device)
                targets = targets.to(device)

                # forward propagation
                outputs = model(batch)

                # calculate the loss
                loss = criterion(outputs, targets)

                # save predictions
                y_pred.extend( outputs.argmax(dim=1).cpu().numpy() )

                # accumulate correct count
                _, preds = torch.max(outputs, 1)
                correct_test += torch.sum(preds == targets.data)


        # Calculate accuracy
        train_acc = correct_train.item() / train_set.data.shape[0]
        test_acc = correct_test.item() / test_set.data.shape[0]

        print('Training accuracy: {:.2f}%'.format(float(train_acc) * 100))
        print('Test accuracy: {:.2f}%\n'.format(float(test_acc) * 100))


    return train_losses, test_losses, y_pred

In [19]:
#@title
train_losses, test_losses, y_pred = train(optimizer, model, num_epochs=num_epochs)

Epoch 1


Loss: 1.6988 50000 / 50000,50000


Training accuracy: 42.37%
Test accuracy: 52.05%

Epoch 2


Loss: 1.1780 50000 / 50000,50000


Training accuracy: 57.60%
Test accuracy: 63.91%

Epoch 3


...

Training accuracy: 87.61%
Test accuracy: 81.84%

Epoch 49


Loss: 0.3447 50000 / 50000,50000


Training accuracy: 87.93%
Test accuracy: 81.75%

Epoch 50


Loss: 0.3395 50000 / 50000,50000


Training accuracy: 87.83%
Test accuracy: 82.49%



여기까지 진행하고! ipynb 파일 제출해주세요.